In [8]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/bus_terminals.csv')
merged_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/merged_matrix.csv')
bus_capacity = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/refs/heads/main/datasets/bus_divisions_cleaned_capacity%20(1).csv')

# VRP

In [9]:
# ============================================================
# 0)  CONFIG
# ============================================================
FILE = "new_matrix.csv"          # distance/time matrix (seconds)
TIME_LIMIT_SEC = 30           # give the solver 3 minutes
MAX_STOPS_PER_ROUTE = 9     # None = unlimited

# exact list & order of depots to use
TERMINALS = ['BD14', 'BD15', 'BT03', 'BL14', 'BD04', 'BT11', 'BT06', 'BL20', 'BT18', 'BL23']

# ============================================================
# 1)  LOAD & CLEAN THE MATRIX
# ============================================================
import pandas as pd, numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

df  = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/new_matrix.csv', index_col=0)                 # IDs in the index
bad = df.apply(pd.to_numeric, errors="coerce")   # force numeric

if bad.isna().any().any():
    r = bad.index[ bad.isna().any(axis=1) ][0]
    c = bad.columns[ bad.isna().any(axis=0) ][0]
    raise ValueError(f"non-numeric cell at row {r!r}, column {c!r}")


dist_mat = df.astype(np.int32).to_numpy()       # seconds
ids      = df.index.astype(str).tolist()
n_nodes  = len(ids)

# ------------------------------------------------------------
# make sure every requested depot exists in the CSV
missing = [d for d in TERMINALS if d not in ids]
assert not missing, f"Depot(s) not found in CSV: {missing}"

# OR-Tools wants the depot indices
bus_idx = [ids.index(d) for d in TERMINALS if d in ids]

# classify the other nodes (not strictly needed, but sanity)
acc_idx   = [i for i,s in enumerate(ids) if s.startswith("A")]
venue_idx = [i for i,s in enumerate(ids) if s.startswith("V")]
assert acc_idx and venue_idx, "Need at least one A- and one V- node"

# ============================================================
# 2)  ROUTING MODEL
# ============================================================
man = pywrapcp.RoutingIndexManager(n_nodes, len(bus_idx), bus_idx, bus_idx)
rt  = pywrapcp.RoutingModel(man)

# transit (cost) = travel seconds
def sec_cb(fi, ti):
    f, t = man.IndexToNode(fi), man.IndexToNode(ti)
    return int(dist_mat[f][t])

transit = rt.RegisterTransitCallback(sec_cb)
rt.SetArcCostEvaluatorOfAllVehicles(transit)

# limit total stops if desired
if MAX_STOPS_PER_ROUTE:
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, MAX_STOPS_PER_ROUTE, True, "Stops")
else:
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, 1000, True, "Stops")  # Arbitrary upper limit

# ---- force every bus to be used (≥ 1 real stop) ---------------------------
if MAX_STOPS_PER_ROUTE:
    stop_dim = rt.GetDimensionOrDie("Stops")
else:
    # create a tiny 1-per-hop dimension just for this purpose
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, 10**6, True, "MustUse")
    stop_dim = rt.GetDimensionOrDie("MustUse")

# ---- ensure at least one venue per route ---------------------------
venue_indices = [man.NodeToIndex(i) for i in venue_idx]
for vehicle_id in range(len(bus_idx)):
    # Add a constraint to ensure at least one venue is visited per route
    venue_visited = [rt.NextVar(venue) for venue in venue_indices]
    rt.solver().Add(rt.solver().Sum(venue_visited) > 1)

#--- set minimum stops per route ---------------------------
stops_dimension = rt.GetDimensionOrDie("Stops")
for vehicle_id in range(len(TERMINALS)):
    end_index = rt.End(vehicle_id)
    stops_dimension.CumulVar(end_index).SetMin(3)  # Set minimum useful length


# ============================================================
# 3)  SEARCH PARAMETERS
# ============================================================
p = pywrapcp.DefaultRoutingSearchParameters()
p.first_solution_strategy    = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
p.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
p.time_limit.seconds         = TIME_LIMIT_SEC

solution = rt.SolveWithParameters(p)

# ============================================================
# 4)  SINGLE-LINE REPORT  (seconds → minutes)
# ============================================================
if not solution:
    print("❌  No solution found — try increasing TIME_LIMIT_SEC "
          "or relaxing MAX_STOPS_PER_ROUTE.")
else:
    for v, depot_node in enumerate(bus_idx):
        idx = rt.Start(v)
        route = [ids[depot_node]]          # start depot
        time_sec = 0

        while not rt.IsEnd(idx):
            prev = idx
            idx  = solution.Value(rt.NextVar(idx))
            route.append(ids[man.IndexToNode(idx)])
            time_sec += dist_mat[man.IndexToNode(prev), man.IndexToNode(idx)]

        stops = len(route) - 2              # minus start & return depot
        minutes = time_sec / 60
        print(f"{ids[depot_node]:<5}: " + " → ".join(route)
              + f"   ({stops} stops, {minutes:.1f} min)")

BD14 : BD14 → V30 → BT08 → A41 → A34 → A33 → V17 → A12 → A13 → BD14   (8 stops, 76.1 min)
BD15 : BD15 → A1 → A19 → A21 → A4 → A3 → A2 → BD15   (6 stops, 76.1 min)
BT03 : BT03 → A16 → A9 → A30 → A42 → A38 → A24 → A35 → BT03   (7 stops, 235.7 min)
BL14 : BL14 → A15 → A40 → A39 → A10 → A25 → A6 → V12 → BL14   (7 stops, 123.0 min)
BD04 : BD04 → V31 → A14 → BT24 → A8 → A18 → V9 → A17 → BD04   (7 stops, 107.0 min)
BT11 : BT11 → A43 → BT16 → A23 → V24 → V14 → A31 → BT11   (6 stops, 68.7 min)
BT06 : BT06 → A28 → A27 → V5 → A29 → A26 → BT06   (5 stops, 52.3 min)
BL20 : BL20 → A51 → V8 → A49 → A48 → A47 → A50 → V11 → A46 → BL20   (8 stops, 64.2 min)
BT18 : BT18 → A22 → A20 → V28 → A44 → A7 → V18 → A37 → BT21 → BT18   (8 stops, 92.9 min)
BL23 : BL23 → A5 → V19 → BD16 → A32 → A11 → A36 → A45 → V20 → BL23   (8 stops, 169.5 min)


### node_ids

In [10]:
# Extract the first column (index) from new_matrix
node_ids = df.index.tolist()

# Initialize an empty list to store the coordinates
coordinates = []

# Iterate through the node IDs and match them with the corresponding coordinates
for node_id in node_ids:
    if node_id in venues['id'].values:
        # Match with venues
        venue_row = venues[venues['id'] == node_id]
        coordinates.append([node_id, venue_row['Latitude'].values[0], venue_row['Longitude'].values[0]])
    elif node_id in accomodations_clusters['id'].values:
        # Match with accommodation clusters
        accom_row = accomodations_clusters[accomodations_clusters['id'] == node_id]
        coordinates.append([node_id, accom_row['avg_latitude'].values[0], accom_row['avg_longitude'].values[0]])
    elif node_id in bus_terminals['id'].values:
        # Match with bus terminals
        terminal_row = bus_terminals[bus_terminals['id'] == node_id]
        coordinates.append([node_id, terminal_row['Latitude'].values[0], terminal_row['Longitude'].values[0]])
    else:
        # If no match is found, append NaN for coordinates
        coordinates.append([node_id, float('nan'), float('nan')])

# Create the node_coords dataframe
node_coords = pd.DataFrame(coordinates, columns=['Node ID', 'Latitude', 'Longitude'])

# Display the resulting dataframe
node_coords

,Node ID,Latitude,Longitude
0,BD14,33.993948,-118.476437
1,BD15,34.085197,-118.382010
2,BD16,34.236880,-118.598039
3,BT16,34.041206,-118.187315
4,BT03,34.201350,-118.450750
...,...,...,...
75,A47,34.091356,-118.279988
76,A48,34.063247,-118.298355
77,A49,34.054311,-118.280299
78,A50,34.083647,-118.255568


In [11]:
# Add a new column 'Type' to node_coords based on the first letter of 'Node ID'
node_coords['Type'] = node_coords['Node ID'].apply(
    lambda x: 'Depot' if x.startswith('B') else 
              'Accommodation' if x.startswith('A') else 
              'Venue' if x.startswith('V') else 'Unknown'
)

# Display the updated dataframe
node_coords

,Node ID,Latitude,Longitude,Type
0,BD14,33.993948,-118.476437,Depot
1,BD15,34.085197,-118.382010,Depot
2,BD16,34.236880,-118.598039,Depot
3,BT16,34.041206,-118.187315,Depot
4,BT03,34.201350,-118.450750,Depot
...,...,...,...,...
75,A47,34.091356,-118.279988,Accommodation
76,A48,34.063247,-118.298355,Accommodation
77,A49,34.054311,-118.280299,Accommodation
78,A50,34.083647,-118.255568,Accommodation


In [12]:
node_ids = node_coords['Node ID'].tolist()
node_id_to_coords = dict(zip(node_coords['Node ID'], zip(node_coords['Latitude'], node_coords['Longitude'])))
node_id_to_type = dict(zip(node_coords['Node ID'], node_coords['Type']))


### map!

In [13]:
import folium
from folium import PolyLine


# Map center around LA
la_center = [34.0522, -118.2437]
m = folium.Map(location=la_center, zoom_start=11)

# Color mapping for node types
type_colors = {
    'Depot': 'red',
    'Venue': 'green',
    'Accommodation': 'blue'
}

# Emoji mapping for tooltip fun
type_emojis = {
    'Depot': '🚌',
    'Venue': '🏟️',
    'Accommodation': '🏨'
}

# Add markers for all nodes
for node_id in node_ids:
    coord = node_id_to_coords.get(node_id)
    if coord is None:
        continue  # skip if coordinates missing
    
    node_type = node_id_to_type.get(node_id, 'Accommodation')  # default to Accommodation
    color = type_colors.get(node_type, 'gray')
    emoji = type_emojis.get(node_type, '')
    
    folium.CircleMarker(
        location=coord,
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.9,
        tooltip=f"{emoji} {node_id} ({node_type})"
    ).add_to(m)

# Colors for routes
route_colors = ['purple', 'orange', 'darkred', 'cadetblue', 'darkblue', 'blue', 'gray', 'darkgreen']

# Plot the routes from your solver solution
if solution:
    for v, depot_node in enumerate(bus_idx):
        idx = rt.Start(v)
        route_coords = []
        
        while not rt.IsEnd(idx):
            node = man.IndexToNode(idx)
            node_id = ids[node]
            coord = node_id_to_coords.get(node_id)
            if coord:
                route_coords.append(coord)
            idx = solution.Value(rt.NextVar(idx))
            
        # Add last depot coord to close the route
        end_node = man.IndexToNode(idx)
        end_node_id = ids[end_node]
        end_coord = node_id_to_coords.get(end_node_id)
        if end_coord:
            route_coords.append(end_coord)
        
        # Draw polyline for route if it has at least 2 points
        if len(route_coords) > 1:
            folium.PolyLine(
                route_coords,
                color=route_colors[v % len(route_colors)],
                weight=5,
                opacity=0.7,
                tooltip=f"Bus route {v+1} (Depot {ids[depot_node]})"
            ).add_to(m)
else:
    print("No solution found to plot.")


m

In [14]:
metro = pd.read_csv('/Users/leonardogreco/Documents/EPFL/M2/Operations/SLO_LA_Olympics/datasets/Metro.csv')
# Create a map centered around Los Angeles
# Add markers for each metro station
for _, row in metro.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        fill_opacity=0.9,
        popup=f"ID: {row['stop_id']}<br>Stop: {row['stop_name']}",
        color='magenta',
        fill_color='cyan',
    ).add_to(m)
# Display the map
m


In [15]:
# Extract nodes from bus route 3
route_3_nodes = []
bus_index = 2  # Bus route 3 corresponds to index 2

if solution:
    idx = rt.Start(bus_index)
    while not rt.IsEnd(idx):
        node = man.IndexToNode(idx)
        route_3_nodes.append(ids[node])
        idx = solution.Value(rt.NextVar(idx))
    # Add the last depot node to close the route
    route_3_nodes.append(ids[man.IndexToNode(idx)])

route_3_nodes

['BT03', 'A16', 'A9', 'A30', 'A42', 'A38', 'A24', 'A35', 'BT03']

In [16]:
# Extract nodes from bus route 5
route_5_nodes = []
bus_index = 4  # Bus route 5 corresponds to index 4

if solution:
    idx = rt.Start(bus_index)
    while not rt.IsEnd(idx):
        node = man.IndexToNode(idx)
        route_5_nodes.append(ids[node])
        idx = solution.Value(rt.NextVar(idx))
    # Add the last depot node to close the route
    route_5_nodes.append(ids[man.IndexToNode(idx)])

route_5_nodes

['BD04', 'V31', 'A14', 'BT24', 'A8', 'A18', 'V9', 'A17', 'BD04']

## Analysis of inital VRP

### CO2 Impact estimation

In [17]:
# Calculate the total time for all routes
total_route_time = 0

if solution:
    for v, depot_node in enumerate(bus_idx):
        idx = rt.Start(v)
        route_time = 0
        
        while not rt.IsEnd(idx):
            prev = idx
            idx = solution.Value(rt.NextVar(idx))
            route_time += dist_mat[man.IndexToNode(prev), man.IndexToNode(idx)]
        
        total_route_time += route_time

# Convert total time to minutes
total_route_time_minutes = total_route_time / 60
print(f"Total Route Time: {total_route_time_minutes:.2f} minutes")

Total Route Time: 1065.53 minutes


In [18]:
# Constants for CO₂ emissions (in grams CO₂ per km per bus)
EMISSION_DIESEL = 1050   # baseline for traditional buses (grams/km)
EMISSION_ELECTRIC = 0    # assuming zero emissions for electric buses

# Average speed of a bus in km/h
AVERAGE_SPEED_KMPH = 25  # adjust based on traffic assumptions in LA

# Assuming route_times is a list of route durations in hours
# e.g., [1.2, 0.75, 0.5] meaning 1.2h, 45min, 30min
total_time_hours = total_route_time_minutes / 60.

# Convert total time to estimated total distance using average speed
total_distance = total_time_hours * AVERAGE_SPEED_KMPH

# CO₂ Emissions
co2_emissions_diesel = total_distance * EMISSION_DIESEL
co2_emissions_electric = total_distance * EMISSION_ELECTRIC
co2_saved = co2_emissions_diesel - co2_emissions_electric

# Output
print(f"Total Time Traveled: {total_time_hours:.2f} hours")
print(f"Estimated Total Distance: {total_distance:.2f} km (assuming {AVERAGE_SPEED_KMPH} km/h)")
print(f"CO₂ Emissions with Diesel Buses: {co2_emissions_diesel/1000:.2f} kg")
print(f"CO₂ Emissions with Electric Buses: {co2_emissions_electric:.2f} g")
print(f"CO₂ Saved: {co2_saved/1000:.2f} kg")


Total Time Traveled: 17.76 hours
Estimated Total Distance: 443.97 km (assuming 25 km/h)
CO₂ Emissions with Diesel Buses: 466.17 kg
CO₂ Emissions with Electric Buses: 0.00 g
CO₂ Saved: 466.17 kg


# CVRP

### generate demand list

In [19]:
# Extract the total accommodates for accommodation clusters
accommodation_demand = accomodations_clusters[['id', 'total_accommodates']].rename(columns={'total_accommodates': 'Demand'})

# Extract the approximate capacity for venues
venue_demand = venues[['id', 'Approx. Capacity']].rename(columns={'Approx. Capacity': 'Demand'})

# Concatenate both dataframes
demand_df = pd.concat([accommodation_demand, venue_demand], ignore_index=True)

# Ensure the Demand column is numeric
demand_df['Demand'] = pd.to_numeric(demand_df['Demand'], errors='coerce')

# Display the resulting dataframe
demand_df

,id,Demand
0,A1,6969.0
1,A2,6395.0
2,A3,2801.0
3,A4,7026.0
4,A5,4909.0
...,...,...
80,V30,8000.0
81,V31,5000.0
82,V32,10000.0
83,V33,2000.0


In [20]:
# Convert demand_df to a dictionary: {"A1": 6969.0, ..., "V33": 2000.0}
demand_map = demand_df.set_index("id")["Demand"].fillna(0).to_dict()

# Match demand to matrix order (fill 0 if ID is missing)
demand_list = [int(demand_map.get(id_, 0)) for id_ in ids]


### create df for bus capacity --> to fix w matching column

In [21]:
bus_terminals

,FACILITY,id,NAME,Latitude,Longitude
0,TERMINAL,BT25,117th ST. / FIGUEROA - BUS LAY OVER,33.927357,-118.282588
1,TERMINAL,BT07,18TH ST. - BUS LAYOVER,34.032369,-118.264098
2,TERMINAL,BT13,6th & WILTON - BUS LAYOVER,34.064388,-118.313779
3,TERMINAL,BT10,85th & CENTRAL - BUS LAYOVER,33.961078,-118.256417
4,DIVISION,BD01,ACTIVE BUS OPERATING DIVISION,34.038218,-118.239042
...,...,...,...,...,...
64,TERMINAL,BT19,U.S.C. MEDICAL CENTER BUSWAY STATION,34.056539,-118.211368
65,LOCATION,BL03,VERNON YARD,34.004204,-118.226913
66,LOCATION,BL07,WAYSIDE RAIL COMMUNICATIONS,34.071472,-118.227312
67,TERMINAL,BT05,WEST L..A. TRANSIT CENTER,34.034825,-118.365077


In [22]:
# Create a copy of bus_capacity
bus_capacity_copy = bus_capacity.copy()

# Assuming bus_divisions is a DataFrame with 'id' column
# Match the id from bus_divisions and add it to the copy of bus_capacity
bus_capacity_copy['id'] = bus_terminals['id']

In [23]:
# List of IDs to keep
ids_to_keep = TERMINALS

# Filter the DataFrame
bus_capacity_copy = bus_capacity_copy[bus_capacity_copy['id'].isin(ids_to_keep)]
bus_capacity_copy


,FACILITY,4Digits,NAME,Bus Type,Size (in foot),Passenger Capacity,Amonut,Address,City,State,Zip,Full Address,Latitude,Longitude,id
11,DIVISION,7,ACTIVE BUS OPERATING DIVISION,CNG-powered,60,60,5,8800 SANTA MONICA BLVD.,LOS ANGELES,CA,90291,8800 SANTA MONICA BLVD. LOS ANGELES CA 90291,34.085197,-118.382010,BD14
12,DIVISION,8,ACTIVE BUS OPERATING DIVISION,CNG-powered,40,40,61,9201 CANOGA AVE.,CHATSWORTH,CA,90069,9201 CANOGA AVE. CHATSWORTH CA 90069,34.236880,-118.598039,BD15
23,DIVISION,18,ACTIVE BUS OPERATING DIVISION,CNG-powered,60,60,62,450 W. GRIFFITH ST.,GARDENA,CA,90014,450 W. GRIFFITH ST. GARDENA CA 90014,33.862207,-118.279704,BT03


In [24]:
['BD14', 'BD15', 'BT03', 'BL14', 'BD04', 'BT11', 'BT06', 'BL20', 'BT18', 'BL23']

['BD14',
 'BD15',
 'BT03',
 'BL14',
 'BD04',
 'BT11',
 'BT06',
 'BL20',
 'BT18',
 'BL23']

### split nodes from demand_list

In [25]:
import numpy as np
import pandas as pd

def split_high_demand_nodes_time_matrix(time_matrix, ids, demand_df, capacity):
    # Map of original ID → demand (NaN → 0)
    demand_map = demand_df.set_index("id")["Demand"].fillna(0).to_dict()
    
    new_ids = []
    new_demands = []
    mapping = {}  # old_id -> list of new IDs

    new_rows = []

    for idx, old_id in enumerate(ids):
        demand = demand_map.get(old_id, 0)
        if demand <= capacity:
            # Keep node as-is
            new_ids.append(old_id)
            new_demands.append(int(demand))
            mapping[old_id] = [old_id]
            new_rows.append(time_matrix[idx])
        else:
            # Split node into parts
            num_splits = int(np.ceil(demand / capacity))
            split_demand = int(np.ceil(demand / num_splits))
            mapping[old_id] = []
            for s in range(num_splits):
                new_id = f"{old_id}_{s+1}"
                new_ids.append(new_id)
                new_demands.append(split_demand)
                mapping[old_id].append(new_id)
                new_rows.append(time_matrix[idx])

    # Now replicate columns in time_matrix for the split nodes
    new_matrix = np.array(new_rows)
    expanded_cols = []
    for idx, old_id in enumerate(ids):
        demand = demand_map.get(old_id, 0)
        if demand <= capacity:
            expanded_cols.append(new_matrix[:, idx])
        else:
            num_splits = int(np.ceil(demand / capacity))
            expanded_cols.extend([new_matrix[:, idx]] * num_splits)

    final_matrix = np.column_stack(expanded_cols)

    return final_matrix.astype(int), new_ids, new_demands, mapping


In [26]:
type(time_matrix), time_matrix.shape


(pandas.core.frame.DataFrame, (85, 86))

In [27]:
time_matrix

,Unnamed: 0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,A42,A43,A44,A45,A46,A47,A48,A49,A50,A51
0,V1,0.0,NaN,2540.0,497.0,1185.0,1961.0,2113.0,497.0,1840.0,...,4051.0,814.0,1683.0,2703.0,714.0,1086.0,1064.0,714.0,1003.0,563.0
1,V2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V3,2748.0,NaN,0.0,2572.0,2685.0,2454.0,1706.0,2572.0,1999.0,...,5643.0,2077.0,3248.0,4462.0,2390.0,2816.0,2940.0,2672.0,2733.0,2559.0
3,V4,491.0,NaN,2421.0,0.0,1321.0,2097.0,2249.0,0.0,1976.0,...,3747.0,735.0,1379.0,2469.0,439.0,783.0,789.0,346.0,699.0,226.0
4,V5,1407.0,NaN,2851.0,1454.0,0.0,2310.0,2462.0,1454.0,2192.0,...,4182.0,1772.0,2320.0,2700.0,1672.0,2044.0,2052.0,1672.0,1961.0,1521.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,A47,1040.0,NaN,2722.0,836.0,1870.0,2582.0,2790.0,836.0,2465.0,...,3537.0,1007.0,1228.0,2259.0,687.0,0.0,722.0,795.0,564.0,858.0
81,A48,1032.0,NaN,2902.0,806.0,1943.0,2719.0,2871.0,806.0,2598.0,...,3637.0,1187.0,1328.0,2359.0,883.0,705.0,0.0,458.0,969.0,831.0
82,A49,667.0,NaN,2597.0,373.0,1497.0,2273.0,2426.0,373.0,2153.0,...,3816.0,874.0,1507.0,2538.0,592.0,719.0,458.0,0.0,889.0,396.0
83,A50,948.0,NaN,2629.0,744.0,1778.0,2490.0,2698.0,744.0,2373.0,...,3644.0,915.0,1255.0,2367.0,595.0,560.0,916.0,840.0,0.0,766.0


In [28]:
ids

['BD14',
 'BD15',
 'BD16',
 'BT16',
 'BT03',
 'BT08',
 'BT24',
 'BL14',
 'BD04',
 'BT11',
 'BT06',
 'BT21',
 'BL20',
 'BT18',
 'BL23',
 'V9',
 'V30',
 'V5',
 'V31',
 'V17',
 'V14',
 'V24',
 'V8',
 'V11',
 'V12',
 'V19',
 'V18',
 'V28',
 'V20',
 'A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'A10',
 'A11',
 'A12',
 'A13',
 'A14',
 'A15',
 'A16',
 'A17',
 'A18',
 'A19',
 'A20',
 'A21',
 'A22',
 'A23',
 'A24',
 'A25',
 'A26',
 'A27',
 'A28',
 'A29',
 'A30',
 'A31',
 'A32',
 'A33',
 'A34',
 'A35',
 'A36',
 'A37',
 'A38',
 'A39',
 'A40',
 'A41',
 'A42',
 'A43',
 'A44',
 'A45',
 'A46',
 'A47',
 'A48',
 'A49',
 'A50',
 'A51']

In [29]:
print(ids[:5])  # ['A1', 'A2', 'V1', ...]
print(len(ids), time_matrix.shape[0])  # both should match


['BD14', 'BD15', 'BD16', 'BT16', 'BT03']
80 85


In [30]:
# Apply the transformation
time_matrix_split, ids_split, demand_list_split, id_mapping = split_high_demand_nodes_time_matrix(
    time_matrix=time_matrix,
    ids=ids,
    demand_df=demand_df,
    capacity=50
)

KeyError: 0

In [ ]:
print(f"Original nodes: {len(ids)}")
print(f"Transformed nodes: {len(ids_split)}")
print(f"Expanded matrix shape: {time_matrix_split.shape}")


### run cvrp

In [ ]:
# Demand & vehicle config

bus_capacity = 50
num_buses = len(bus_idx) ## use random number for the moment, try to integrate collected data later

# Routing index manager
man = pywrapcp.RoutingIndexManager(n_nodes, num_buses, bus_idx, bus_idx)
rt = pywrapcp.RoutingModel(man)

# Time callback (in seconds)
def sec_cb(from_index, to_index):
    f, t = man.IndexToNode(from_index), man.IndexToNode(to_index)
    return int(dist_mat[f][t])
transit = rt.RegisterTransitCallback(sec_cb)
rt.SetArcCostEvaluatorOfAllVehicles(transit)

# Capacity callback
def demand_cb(from_index):
    node = man.IndexToNode(from_index)
    return demand_list[node]
demand_idx = rt.RegisterUnaryTransitCallback(demand_cb)

rt.AddDimensionWithVehicleCapacity(
    demand_idx,
    0,
    [bus_capacity] * num_buses,
    True,
    "Capacity"
)

True

In [ ]:
from ortools.constraint_solver import routing_enums_pb2

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.time_limit.seconds = 30  # Set a time limit for the solver

# Solve the problem
solution = rt.SolveWithParameters(search_parameters)

# Extract and print routes
if solution:
    print("Solution found!")
    for vehicle_id in range(num_buses):
        index = rt.Start(vehicle_id)
        route = []
        route_demand = 0
        while not rt.IsEnd(index):
            node = man.IndexToNode(index)
            route.append(node)
            route_demand += demand_list[node]
            index = solution.Value(rt.NextVar(index))
        route.append(man.IndexToNode(index))  # Add the end node
        print(f"Route for bus {vehicle_id}: {route}")
        print(f"Total demand served by bus {vehicle_id}: {route_demand}")
else:
    print("No solution found.")

No solution found.


In [ ]:
max(demand_list) <= bus_capacity


False

In [ ]:
# Add min 3 stops per route
#stops_dim = rt.GetDimensionOrDie("Stops")
#for vehicle_id in range(num_buses):
    #end_idx = rt.End(vehicle_id)
    #stops_dim.CumulVar(end_idx).SetMin(3)

# Ensure each route includes at least one venue
#venue_indices = [man.NodeToIndex(i) for i in venue_idx]
#for vehicle_id in range(num_buses):
    #rt.solver().Add(rt.solver().Sum([rt.NextVar(i) for i in venue_indices]) > 1)


In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# ============================================================
# DEMAND & VEHICLE CONFIGURATION
# ============================================================
# Example demand list: [0, 3, 2, 0, 5, ...]  # 0 for depots, >0 for A/V nodes
# You must define this externally to match your matrix order.
demand_list = [0 if id_ in TERMINALS else your_demand_map.get(id_, 0) for id_ in ids]
bus_capacity = 50  # example capacity, adjust as needed
num_buses = len(bus_idx)

# ============================================================
# ROUTING SETUP
# ============================================================
man = pywrapcp.RoutingIndexManager(n_nodes, num_buses, bus_idx, bus_idx)
rt = pywrapcp.RoutingModel(man)

# Travel time callback (in seconds)
def sec_cb(from_index, to_index):
    f, t = man.IndexToNode(from_index), man.IndexToNode(to_index)
    return int(dist_mat[f][t])
transit = rt.RegisterTransitCallback(sec_cb)
rt.SetArcCostEvaluatorOfAllVehicles(transit)

# Capacity/demand callback
def demand_cb(from_index):
    node = man.IndexToNode(from_index)
    return demand_list[node]
demand_idx = rt.RegisterUnaryTransitCallback(demand_cb)

# Add capacity constraint
rt.AddDimensionWithVehicleCapacity(
    demand_idx,
    0,  # no slack
    [bus_capacity] * num_buses,
    True,  # start at zero
    "Capacity"
)

# ============================================================
# OPTIONAL: STOPS CONSTRAINTS
# ============================================================
ones_cb = rt.RegisterUnaryTransitCallback(lambda _: 1)
rt.AddDimension(ones_cb, 0, MAX_STOPS_PER_ROUTE or 1000, True, "Stops")
stops_dim = rt.GetDimensionOrDie("Stops")

# Force minimum 3 stops (non-trivial routes)
for vehicle_id in range(num_buses):
    end_idx = rt.End(vehicle_id)
    stops_dim.CumulVar(end_idx).SetMin(3)

# ============================================================
# VENUE CONSTRAINT (at least one per route)
# ============================================================
venue_indices = [man.NodeToIndex(i) for i in venue_idx]
for vehicle_id in range(num_buses):
    rt.solver().Add(rt.solver().Sum([rt.NextVar(i) for i in venue_indices]) > 1)

# ============================================================
# SEARCH PARAMETERS
# ============================================================
p = pywrapcp.DefaultRoutingSearchParameters()
p.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
p.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
p.time_limit.seconds = TIME_LIMIT_SEC

solution = rt.SolveWithParameters(p)

# ============================================================
# REPORTING
# ============================================================
if not solution:
    print("❌ No solution found. Try adjusting time limit or capacity.")
else:
    for v in range(num_buses):
        idx = rt.Start(v)
        route = [ids[man.IndexToNode(idx)]]
        time_sec = 0
        load = 0

        while not rt.IsEnd(idx):
            prev_idx = idx
            idx = solution.Value(rt.NextVar(idx))
            node = man.IndexToNode(idx)
            route.append(ids[node])
            time_sec += dist_mat[man.IndexToNode(prev_idx)][node]
            load += demand_list[node]

        stops = len(route) - 2
        minutes = time_sec / 60
        print(f"{ids[man.IndexToNode(rt.Start(v))]:<5}: {' → '.join(route)}   "
              f"({stops} stops, {minutes:.1f} min, {load} passengers)")


NameError: name 'your_demand_map' is not defined